In [ ]:
import numpy as np
import torch
import math
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import GPyOpt
import GPy
from turbo import TurboM
from turbo import Turbo1


import os

import matplotlib as mpl
import matplotlib.tri as tri

import ternary
import pickle
import datetime

from collections import Counter
import matplotlib.ticker as ticker

from sklearn import preprocessing

import pyDOE
import random
from scipy.stats import norm

from sklearn.ensemble import RandomForestRegressor



from sklearn.datasets import load_boston
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import copy

import pickle


import random

import imageio

from scipy.spatial import Delaunay



import time


import tqdm
import gpytorch

from torch.distributions import Normal
from itertools import product

import matplotlib.font_manager as font_manager
from smt.sampling_methods import LHS, Random

In [ ]:
# X: biomass concentration
# g/L
# X_0 = 0.1

# S:substrate concentration
# g/L
# S_0 = 0.1

# CL:dissolved oxygen concentration

# P: penicillin concentration
# g/L
# P_0 = 0


# CO2:carbon dioxide concentration; 

# H:hydrogen ion concentration for pH

# T: temperature.

C_L_star = 8.26

Y_xs = 0.45
Y_xo = 0.04
Y_ps = 0.90
Y_po = 0.20



K_1 = 10**(-10)
K_2 = 7 * 10**(-5)
m_X = 0.014
m_o = 0.467

alpha_1 = 0.143
alpha_2 = 4*10**(-7)
alpha_3 = 10**(-4)
mu_X = 0.092
K_X = 0.15
# K_ox = 2*10**(-2)
# K_op = 5*10**(-4)
mu_p = 0.005
K_p = 0.0002
K_I = 0.10
p = 3
K = 0.04
k_g = 7 * 10**(3)
E_g = 5100
k_d = 10**(33)
E_d = 50000

# rou_dot_C_p = 1/1500
# rou_c_dot_C_pc = 1/2000

rou_dot_C_p = 1000
rou_c_dot_C_pc = 1000


r_q1 = 60
r_q2 = 1.6783 * 10**(-4)
a = 1000
b = 0.60

alpha = 70
beta = 0.4
lambd = 2.5 * 10**(-4)
gamma = 10**(-5)


# kelvin
T_v = 273
T_o = 373



# CAL/(MOL K)
R = 1.9872




# parameters


P = 0 initial

V_max = 180
V = [60, 120] initial

X = [0.01, 12] initial

Q_rxn = 0 initial

system T = 293 - 303 initial range
F_c range [0, 5]

S = [0.1, 18] initial
Feed flow rate F [0.01, 0.50]
Feed substrate concentration s_f range [400, 600]
Feed substrate temperature T_f = 293 - 303

sufficient oxygen

H: pH 5 - 7.5 kept constant

CO2 = 0 initial

t = 0 initial






In [ ]:
# total unit time: hrs
t = 2500

V_max = 180
V_limits = [60, 120]



X_limits = [0.05, 18]


CO2 = 0

T_limits = [293, 303]


S_limits = [0.05, 18]

F_limits = [0.01, 0.50]
s_f_limits = [500, 700]



H_limits = [5, 6.5]


limits = [V_limits, X_limits, T_limits, S_limits, F_limits, s_f_limits, H_limits]


In [ ]:
def penicilin_exp_BO(X_input):
    print(X_input)
    
    V, X, T, S, F, s_f, H_ = X_input[0],X_input[1],X_input[2],X_input[3], X_input[4], X_input[5], X_input[6]
    
    P = 0
    CO2 = 0
    t = 2500
    
    l_P = []
    l_V = []
    l_X = []
    l_T = []
    l_S = []
    l_F = []
    l_s_f = []
    l_H_ = []
    l_CO2 = []
    l_t = []


    l_P.append(P)
    l_V.append(V)
    l_X.append(X)
    l_T.append(T)
    l_S.append(S)
    l_F.append(F)
    l_s_f.append(s_f)
    l_H_.append(H_)
    l_CO2.append(CO2)
    l_t.append(0)

    H = 10**(-H_)

    for i in np.arange(t) + 1:
        
        F_loss = V * lambd*(np.exp(5*((T - T_o)/(T_v - T_o))) - 1)
        dV_dt = F  - F_loss

        mu = (mu_X / (1 + K_1/H + H/K_2)) * (S / (K_X * X + S))  * ((k_g * np.exp(-E_g/(R*T))) - (k_d * np.exp(-E_d/(R*T))))
        dX_dt = mu * X - (X / V) * dV_dt
        
        mu_pp = mu_p * (S / (K_p + S + S**2 / K_I)) 
        dS_dt = - (mu / Y_xs) * X - (mu_pp/ Y_ps) * X - m_X * X + F * s_f / V - (S / V) * dV_dt
        
        dP_dt = (mu_pp * X) - K * P - (P / V) * dV_dt    
        
        dCO2_dt = alpha_1 *dX_dt + alpha_2 * X + alpha_3


        # UPDATE
        P = P + dP_dt
        V = V + dV_dt
        X = X + dX_dt
        S = S + dS_dt
        CO2 = CO2 + dCO2_dt
        
        
        
        l_P.append(P)
        l_V.append(V)
        l_X.append(X)
        l_T.append(T)
        l_S.append(S)
        l_F.append(F)
        l_s_f.append(s_f)
        l_H_.append(H_)
        l_CO2.append(CO2)
        l_t.append(i)

        if V > V_max:
#             print('Too large V')
            break

        if S < 0:
#             print('Too small S')
            break

        if dP_dt < 10e-12:
#             print('Converged P')
            break

#     print('final results: ' + 'P = '+str(np.round(P, 2)) +', S = '+str(np.round(S, 2)) + ', X = ' + str(np.round(X, 2)) + ', V = ' + str(np.round(V, 2)) + ', t = ' + str(i))
#     GpyOpt does minimization only
    print(P)
    return -P






In [ ]:
lowerb = np.array(limits)[:,0]
upperb = np.array(limits)[:,1]
dim = len(lowerb)
print(dim)
assert len(lowerb) == len(upperb)

In [ ]:
lowerb

In [ ]:
upperb

In [ ]:
class penicillin:
    def __init__(self, dim=7):
        self.dim = dim
        self.lb = lowerb
        self.ub = upperb
        
    def __call__(self, x):
#         assert len(x) == self.dim
#         assert x.ndim == 1
#         assert np.all(x <= self.ub) and np.all(x >= self.lb)
        pred_val = penicilin_exp_BO(x)
        return pred_val

In [ ]:
seed_list = [15, 361, 5366, 9485, 8754, 1268, 9914, 8450, 9498, 5181, 1850, 4561, 3579, 9359, 3958, 2005, 6917, 2630, 7210, 1565, 8258, 5267, 7658, 1256, 7511, 4783, 9130, 1870, 962, 3558, 3273, 9770, 1872, 2649, 3725, 6433, 1248, 4856, 9881, 8410, 7137, 8056, 8478, 404, 8299, 1748, 9133, 4210, 5993, 1084, 1047, 5673, 7261, 8370, 3850, 7228, 2356, 5004, 6573, 6919, 5437, 3879, 8421, 2817, 8141, 7277, 8473, 9281, 8217, 8537, 8110, 3530, 2528, 4249, 6473, 5902, 1733, 3562, 6232, 9180, 9581, 4863, 6455, 6267, 397, 6571, 7682, 3655, 7695, 2154, 6157, 6971, 2173, 9005, 2441, 6703, 1639, 8149, 3067, 2846, 2169, 1028, 4480, 8621, 5321, 8092, 2448, 3002, 3640, 252, 7340, 3230, 5219, 5445, 520, 8960, 8561, 1950, 7742, 5925, 7894, 6451, 8327, 6679, 1567, 9964, 221, 7288, 6503, 6733, 3473, 5392, 5780, 7941, 3186, 2358, 8525, 7198, 2108, 8808, 4679, 1798, 3816, 7119, 3341, 452, 7081, 4490, 8964, 5409, 3689, 3374, 8459, 1725, 5356, 941, 1745, 138, 7065, 1700, 6068, 2405, 7856, 8817, 6921, 9942, 7909, 8551, 7599, 1782, 3102, 7489, 9145, 5203, 9243, 5665, 7459, 8400, 4465, 8331, 1222, 2967, 6165, 2063, 4569, 2748, 3068, 8000, 3487, 6941, 3248, 7922, 1255, 2618, 6029, 620, 1367, 3882, 1943, 5862, 3734, 174, 1642, 2565, 6276, 6942, 6643, 6883, 5610, 6575, 3474, 517, 1763, 5033, 5480, 5088, 1766, 804, 9661, 2237, 6250, 7481, 4157, 366, 5455, 7936, 1637, 7163, 6330, 6269, 1291, 7439, 9787, 3993, 4212, 6818, 9652, 3533, 928, 719, 8958, 2662, 4037, 2628, 5675, 6191, 1610, 7399, 3789, 48, 1968, 8677, 9064, 5403, 4695, 7317, 9382, 4873, 9049, 2668, 3669, 899, 1329, 7221, 9343, 3234, 4563, 5809, 4553, 2623, 6106, 4001, 942, 5719, 4114, 9164, 3163, 9751, 8709, 6675, 1767, 9868, 7828, 3809, 8566, 1660, 2135, 5726, 3829, 7493, 3749, 77, 6787, 5530, 3134, 9977, 6182, 8207, 850, 826, 3396, 9934, 1265, 843, 7561, 1720, 5175, 5553, 3554, 5836, 7350, 542, 1328, 601, 2480, 2459, 8026, 6563, 5129, 5901, 9656, 761, 1077, 6627, 5774, 3279, 6653, 5451, 1158, 5450, 4130, 6759, 6246, 1718, 9081, 8953, 743, 4201, 718, 1365, 5838, 1259, 5009, 720, 4619, 6803, 8124, 4072, 8249, 2631, 4147, 9225, 203, 5261, 5128, 6889, 9664, 7354, 9603, 8156, 6055, 4038, 3824, 6272, 1011, 1189, 2289, 5664, 5616, 3793, 5749, 9591, 968, 2530, 9194, 6906, 6721, 4420, 5634, 4819, 1092, 324, 6, 5882, 8999, 5585, 4094, 8368, 8620, 4631, 4310, 2464, 4513, 125, 2622, 9695, 544, 7239, 31, 3275, 7290, 2041, 7820, 4366, 5184, 33, 912, 7470, 1442, 6793, 9648, 2151, 1368, 432, 2865, 499, 3988, 5994, 5257, 5976, 5949, 5680, 4252, 838, 7574, 8845, 22, 3254, 4979, 4428, 5690, 6822, 2840, 6018, 2904, 4160, 5176, 8253, 3617, 7735, 910, 8454, 1103, 2923, 7863, 3389, 3895, 8644, 316, 2760, 4347, 9888, 1179, 3062, 8094, 7526, 3568, 8034, 482, 1892, 5677, 9822, 8989, 3141, 9000, 5354, 5177, 8086, 8652, 3190, 9626, 5542, 6047, 3367, 7309, 8704, 4628, 8375, 5778, 6476, 3961, 2933, 6760, 5095, 2638, 7906, 8131, 5811, 6870, 4260, 1452, 1132, 7245, 3353, 6566, 5757, 1562, 5834, 6661, 960, 4101, 391, 8147, 4445, 8009, 1109, 2258, 2737, 91, 2555, 9340, 4134, 9109, 6274, 8689, 8562, 3800, 2161, 7634, 5159, 668, 2456, 5540, 4965, 6882, 388, 8736, 523, 236, 1201, 5635, 4721, 9932, 5613, 1061, 5348, 6720, 7998, 6195, 5367, 3925, 9482, 5596, 5524, 7844, 251, 1231, 7908, 5889, 8865, 4229, 2600, 6134, 9704, 8743, 9647, 6048, 4775, 3147, 5477, 6040, 8260, 3286, 3704, 6095, 3516, 5499, 5916, 5715, 4462, 9056, 7426, 8968, 4689, 936, 3414, 6673, 5752, 5371, 9925, 4654, 7626, 3492, 2012, 3481, 3123, 3404, 1065, 1514, 9027, 16, 6648, 3909, 6062, 3670, 5697, 6150, 1236, 4568, 1258, 5420, 323, 1022, 7303, 2314, 421, 1899, 1342, 6558, 8666, 1708, 9275, 7321, 1846, 4250, 4915, 6877, 325, 2250, 7443, 479, 3682, 2957, 1491, 1472, 271, 3398, 190, 3364, 5654, 4341, 3591, 8104, 9057, 1829, 1049, 7566, 171, 5870, 7688, 8812, 965, 6738, 6524, 9699, 5178, 6110, 731, 364, 6354, 2829, 6520, 4545, 1085, 2710, 5966, 2481, 3089, 3973, 6647, 782, 6546, 4791, 9685, 6861, 5840, 6613, 2761, 1186, 8388, 7542, 8461, 3981, 3556, 5150, 5347, 3622, 475, 6765, 9549, 4625, 9562, 5815, 139, 9255, 1814, 2615, 2392, 8246, 4643, 5148, 2604, 3094, 505, 7161, 3018, 6534, 6947, 9627, 1089, 1498, 9946, 5158, 5854, 3542, 9502, 2897, 3724, 9553, 8988, 1156, 8906, 3718, 2402, 1198, 1362, 9364, 4204, 3549, 6347, 7122, 5865, 8250, 3876, 7946, 4736, 3320, 328, 6051, 698, 5974, 245, 5277, 145, 5691, 4627, 7661, 9272, 3612, 6026, 3007, 2171, 6600, 1622, 5602, 8481, 3452, 6094, 2939, 302, 1636, 9413, 7421, 5174, 7933, 8268, 2582, 4662, 5456, 561, 8318, 9777, 4159, 5495, 8881, 228, 1706, 7590, 7815, 7393, 8320, 7513, 9330, 7418, 6959, 9908, 129, 2787, 1625, 6151, 5212, 6739, 3409, 4216, 8035, 9945, 8225, 6978, 2694, 6385, 6124, 9991, 6828, 6186, 4174, 2672, 1352, 1412, 9318, 94, 5612, 6694, 3472, 1554, 4802, 6021, 4685, 6020, 8443, 6608, 6271, 2370, 3075, 5791, 1118, 7983, 9706, 4787, 8014, 5001, 6649, 3855, 4325, 8728, 1273, 2518, 8543, 7529, 7078, 1605, 5819, 8381, 9312, 3130, 8942, 4866, 2624, 1626, 6309, 7202, 5016, 5048, 3052, 2468, 4266, 5924, 4876, 7017, 4995, 6767, 8326, 1680, 2077, 3456, 2254, 3844, 8820, 3719, 5793, 6676, 556, 1433, 4706, 5794, 3574, 5239, 6641, 8300, 6430, 1088, 624, 8615, 9451, 7370, 2659, 7297, 2182, 1866, 2927, 6659, 4717, 6023, 7777, 6550, 8161, 3274, 4379, 6581, 6652, 5326, 8893, 898, 4497, 1272, 7992, 3001, 5247, 1449, 5896, 6260, 6597, 9832, 8152, 7640, 49, 6275, 539, 173, 8212, 6698, 7639, 9642, 4834, 1311, 7586, 6511, 7969, 904, 3730, 7892, 7987, 9512, 5084, 4415, 5556, 3757, 8572, 481, 1752, 8619, 9265, 374, 2411, 7572, 4494, 9193, 5241, 5325, 3104, 6289, 3510, 6960, 4708, 9474, 8669, 3253, 6234, 1034, 3076, 1845, 8025, 9719, 1746, 5238, 9808, 8750, 2909, 5683, 3603, 9733, 7903, 6262, 8723, 8698, 6450, 4584, 9515, 7556, 6282, 808, 7094, 7250, 4231, 5713, 6364, 4610, 8323, 3462, 37, 2574, 488, 6169, 2902, 9778, 1542, 222, 9517, 3727, 1000, 2644, 1863, 4681, 809, 1067, 2807, 9743, 5426, 7320, 9726, 9667, 1553, 5578, 3148, 6775, 747, 9279, 9623, 7660, 4739, 4715, 2060, 8489, 8243, 771, 5988, 7606, 1042, 7060, 604, 2347, 7707, 1539, 7260, 3587, 5679, 1376, 5269, 4993, 6323, 67, 9558, 2729, 5973, 2619, 64, 8613, 4592, 4337, 5817, 9379, 1030, 977, 2718, 3288, 5990, 8784, 666, 2294, 8057, 5316, 2106, 254, 4447, 6535, 7282, 5592, 2700, 2452, 3350, 3132, 1336, 3894, 8350, 6646, 26, 4650, 3377, 508, 6133, 8495, 8521, 8186, 675, 2260, 2142, 6409, 2834, 5595, 1716, 7088, 3181, 5074, 1161, 2792, 8764, 5550, 5501, 617, 5643, 1159, 2848, 3904, 7817, 1884, 8405, 5114, 865, 5271, 8004, 887, 710, 8789, 6115, 6461, 9367, 1536, 796, 5483, 3289, 1537, 1781, 8535, 915, 5709, 4455, 9616, 7343, 6122, 5655, 5329, 9101, 8039, 2764, 1279, 6768, 6967, 547, 9931, 1604, 4059, 9715, 860, 6610, 418, 5132, 2343, 3304, 3250, 4431, 4457, 6804, 2393, 4369, 4020, 4302, 7331, 1277, 9830, 950, 6417, 1358, 7208, 1040, 522, 3149, 2526, 5773, 439, 9550, 1638, 7405, 7788, 8846, 2134, 6614, 4360, 2228, 7950, 817, 2914, 717, 1657, 4520, 5850, 2253, 4905, 8420, 7723, 783, 6201, 806, 844, 5080, 6266, 3814, 511, 493, 8646, 4304, 1658, 6771, 9563, 2585, 4193, 6400, 8775, 5204, 2560, 6790, 9499, 3820, 837, 7630, 9202, 7898, 9154, 3482, 1721, 124, 5193, 4016, 2319, 632, 4008, 1251, 9493, 3902, 9788, 6066, 978, 95, 2634, 7982, 7505, 6121, 979, 8417, 3675, 9844, 8515, 5517, 9545, 2094, 8399, 2440, 540, 1552, 1635, 7084, 1958, 3741, 6376, 6166, 1343, 1998, 6179, 1505, 4198, 9441, 4485, 9939, 2588, 2328, 577, 2287, 9860, 5688, 274, 6632, 4984, 5971, 9198, 5291, 3415, 6456, 2028, 3766, 386, 6225, 856, 1719, 3490, 6303, 1214, 78, 2011, 8366, 9043, 7895, 4934, 8223, 4918, 5303, 7234, 581, 8226, 7233, 9252, 1898, 7995, 8317, 6938, 7814, 1760, 1874, 2378, 8448, 2870, 6922, 4452, 5788, 303, 3196, 1920, 4580, 1683, 1351, 5146, 8755, 6879, 8228, 5954, 143, 3537, 9785, 7323, 8516, 363, 9271, 9435, 1791, 3974, 6280, 384, 6297, 8720, 4748, 7240, 8810, 2984, 4738, 6190, 9149, 9128, 8757, 1686, 1386, 7772, 1025, 1732, 1556, 4890, 5465, 4012, 8596, 2557, 5470, 7724, 2310, 1597, 3985, 7498, 494, 1405, 1596, 4120, 3607, 6291, 6681, 8534, 8364, 9773, 8241, 3756, 7816, 663, 863, 9892, 1210, 7295, 8994, 6697, 9159, 8480, 8519, 5229, 2968, 6078, 7510, 2971, 2838, 156, 2670, 8859, 1710, 2017, 755, 6508, 6468, 162, 2880, 7872, 1468, 6293, 4334, 1420, 5144, 6954, 8112, 4826, 5706, 5813, 6878, 6462, 9624, 8465, 9227, 9096, 4460, 9713, 8836, 7728, 5345, 4816, 2290, 4235, 5504, 829, 9232, 8874, 2883, 4702, 2272, 8089, 4548, 6454, 8908, 5864, 9671, 9276, 5956, 9307, 9460, 5027, 7030, 5781, 1467, 3118, 1074, 7131, 735, 99, 5300, 8705, 8530, 7592, 1648, 692, 5000, 6170, 2714, 672, 6951, 3109, 9537, 9740, 2406, 42, 4053, 2988, 8254, 973, 6136, 3830, 9705, 2089, 3025, 8334, 2901, 8936, 6103, 9010, 6657, 6463, 6667, 425, 6352, 2024, 9282, 3090, 5952, 8358, 7827, 9463, 526, 4588, 5365, 5472, 2569, 9285, 9174, 5002, 5753, 3890, 5566, 3198, 2243, 6039, 468, 8037, 8622, 5330, 6069, 4345, 3956, 6259, 3057, 7314, 7332, 2673, 1815, 592, 4434, 218, 4699, 4376, 7700, 3297, 2265, 764, 769, 8579, 3737, 4663, 3246, 7554, 7989, 1245, 5256, 6478, 5771, 5101, 9009, 1066, 2701, 7719, 2422, 3200, 7324, 6123, 9968, 6131, 3366, 4872, 2050, 8129, 5734, 6809, 8055, 4653, 5694, 3726, 1670, 8079, 2274, 3802, 9141, 7763, 4519, 4502, 5764, 7024, 8424, 1785, 7383, 8732, 9783, 6578, 9567, 4358, 497, 5038, 1435, 2871, 1397, 9296, 5984, 8138, 7173, 8166, 8232, 9110, 6413, 7970, 6298, 9586, 5565, 7703, 9870, 5630, 4118, 9221, 5007, 2170, 9693, 3582, 3646, 277, 2692, 4290, 2083, 208, 2248, 9417, 3411, 9104, 4188, 6497, 6750, 3027, 5466, 2225, 9163, 7442, 54, 9031, 4815, 3913, 7588, 5947, 2774, 3778, 6204, 9767, 3195, 7533, 4790, 5375, 3352, 872, 6135, 6574, 7205, 6810, 191, 1419, 3408, 2109, 6708, 5551, 2491, 7018, 1946, 8059, 463, 8284, 1656, 2080, 9938, 8163, 5441, 1266, 9117, 9896, 2001, 5299, 403, 7598, 7669, 7427, 1735, 828, 8794, 3298, 6098, 27, 1951, 5315, 7326, 5005, 815, 1480, 6553, 8536, 7424, 498, 8654, 7960, 7031, 9734, 6242, 1825, 3416, 4869, 122, 6713, 5068, 5036, 969, 8725, 6746, 3317, 4843, 9518, 9319, 9501, 5355, 8756, 8027, 7318, 3345, 2821, 1406, 8132, 9470, 3028, 7665, 5579, 3633, 6812, 2928, 1188, 6705, 243, 6905, 9222, 8307, 9529, 2598, 5687, 4086, 3357, 9758, 8277, 4175, 4875, 9620, 4483, 7096, 781, 5989, 2070, 5397, 9231, 9015, 5137, 5717, 3735, 3605, 7614, 2082, 2515, 4132, 2793, 6791, 9003, 7145, 3686, 1164, 6853, 342, 70, 6370, 102, 6482, 7086, 9520, 4395, 5914, 8744, 4473, 5072, 7548, 5278, 346, 6829, 1509, 6107, 4044, 1538, 5637, 2067, 5516, 4538, 9115, 7633, 5963, 5681, 4928, 4052, 6472, 1409, 8638, 1244, 5432, 4904, 7879, 9401, 5478, 7994, 5260, 8897, 1310, 7782, 97, 825, 7603, 9293, 4639, 8878, 8462, 491, 2469, 2576, 6022, 1859, 8930, 7448, 304, 1598, 2002, 121, 780, 3796, 7014, 9001, 6580, 2818, 1571, 9412, 9728, 8016, 1921, 3174, 6012, 5089, 5698, 3657, 9893, 7573, 1667, 4672, 8330, 2300, 2708, 8740, 9203, 8598, 9086, 4476, 4078, 2078, 9486, 6116, 4583, 5094, 57, 2732, 8125, 3381, 6268, 6527, 2059, 5946, 6100, 658, 7779, 8476, 6709, 9880, 4018, 7116, 7445, 1149, 5098, 1983, 4506, 2915, 1411, 2359, 5173, 5746, 5032, 655, 5953, 1878, 1488, 2053, 9024, 350, 3835, 2685, 2160, 8351, 6692, 7377, 7299, 5386, 6332, 9800, 9798, 582, 9630, 1885, 9152, 9347, 2755, 2177, 1516, 1218, 1851, 3746, 6217, 1341, 5856, 3979, 8422, 7891, 2687, 3182, 6740, 945, 8464, 8383, 2983, 2214, 6794, 8961, 8803, 5755, 9557, 4029, 5604, 9570, 4755, 3782, 6036, 6891, 7907, 1246, 9083, 9287, 3987, 3402, 7019, 8633, 9590, 6748, 5416, 9827, 9653, 2639, 8509, 4825, 7047, 5245, 1886, 3040, 4973, 696, 2985, 9195, 5085, 2907, 6200, 5564, 1532, 1853, 4177, 7176, 5039, 1392, 3302, 3928, 9105, 6350, 703, 4251, 8655, 8354, 8745, 9289, 1566, 9069, 8032, 1162, 7972, 1582, 3999, 2860, 1069, 3019, 7962, 3886, 9478, 1490, 2016, 3593, 7832, 9422, 2698, 2740, 2187, 1004, 9430, 484, 6064, 7316, 161, 2782, 9963, 9226, 3112, 1561, 6875, 9815, 9750, 5453, 8998, 5763, 8504, 6102, 1346, 63, 3203, 8871, 9979, 5514, 6338, 5307, 9948, 9366, 670, 4372, 1786, 7624, 7313, 9674, 867, 69, 6254, 7382, 9527, 3866, 6944, 6168, 3772, 8264, 5747, 2249, 73, 8101, 1494, 6855, 4060, 8096, 354, 5800, 7389, 9905, 8851, 8045, 6372, 6556, 6770, 5738, 4092, 1726, 177, 4165, 2771, 9230, 7289, 7440, 3438, 4762, 2285, 6061, 7388, 7099, 258, 2323, 3769, 9065, 1621, 8369, 5849, 9585, 4412, 9561, 2858, 9535, 1111, 3326, 3188, 1083, 3161, 1171, 2805, 3362, 5243, 4646, 2966, 8980, 9666, 276, 7975, 1665, 7391, 6888, 9048, 6279, 9368, 9807, 1023, 2609, 2299, 6988, 8272, 8618, 292, 4333, 247, 4148, 349, 9473, 1145, 4367, 9730, 4213, 4004, 9971, 6622, 6004, 47, 2970, 9954, 5890, 180, 724, 7910, 6405, 4287, 810, 2316, 1191, 6030, 8697, 1776, 6894, 4966, 3392, 4867, 2996, 3968, 7428, 2875, 5481, 201, 7649, 9016, 4238, 6684, 2876, 2994, 2511, 1372, 3805, 6448, 5975, 1297, 6015, 5922, 1027, 3016, 7089, 5847, 8798, 799, 2252, 5502, 309, 9240, 773, 982, 5423, 2906, 4709, 6903, 5858, 8816, 4400, 8587, 4718, 5087, 4621, 4069, 5964, 8386, 8563, 3467, 5380, 7683, 1690, 1399, 5427, 6744, 628, 7121, 6163, 6167, 2788, 1313, 9241, 503, 2605, 7291, 6229, 8195, 148, 1337, 7160, 9615, 3256, 9178, 25, 294, 3532, 7926, 8776, 6206, 9264, 3327, 8608, 583, 9374, 2371, 4320, 1640, 412, 7847, 9540, 3767, 2163, 3111, 8921, 9445, 7076, 6560, 9192, 7232, 1315, 5640, 964, 6561, 1826, 5327, 9794, 8835, 1146, 6161, 2648, 1713, 7570, 5040, 8052, 3893, 9320, 8298, 6691, 41, 7953, 5003, 6745, 5561, 6408, 9902, 4792, 7635, 8203, 204, 674, 6253, 8902, 9327, 3691, 3243, 6587, 5293, 319, 3500, 3219, 6986, 5438, 8338, 5394, 3343, 6852, 1270, 9384, 7597, 4126, 196, 6322, 3336, 5605, 9346, 6729, 9242, 3950, 7840, 8192, 332, 3629, 1865, 9802, 9160, 5372, 2603, 5724, 7878, 9560, 5140, 2341, 3531, 6842, 4954, 4006, 1306, 29, 9124, 8017, 4605, 9455, 1912, 7263, 2974, 606, 821, 8084, 5727, 1010, 6530, 8416, 5802, 7540, 5454, 4827, 976, 4486, 3128, 9261, 8455, 9817, 5280, 516, 4080, 6965, 4276, 5263, 4464, 3330, 2149, 5827, 4542, 576, 7862, 1122, 8487, 5190, 905, 2960, 605, 7701, 645, 6929, 6626, 4321, 7395, 5462, 9672, 5111, 610, 8291, 6630, 9571, 3742, 1283, 819, 5663, 7378, 1669, 5897, 1216, 50, 8559, 8210, 682, 7111, 5880, 7296, 1900, 6562, 2414, 3347, 7744, 9940, 189, 2455, 7507, 3283, 1271, 5740, 3026, 9426, 4254, 2835, 45, 5237, 8447, 2808, 7438, 3659, 7138, 5281, 7747, 3484, 9108, 2806, 6860, 6214, 7978, 4191, 518, 5861, 2246, 2587, 4391, 5071, 4674, 2470, 568, 5543, 9472, 5986, 1910, 3012, 622, 9936, 2741, 8604, 9121, 5006, 1611, 760, 235, 5893, 2112, 2035, 9239, 9363, 5413, 2148, 3215, 6177, 1237, 3905, 6998, 3951, 353, 1867, 8648, 335, 9126, 7500, 5395, 4406, 7579, 5686, 8685, 7384, 7380, 5043, 7360, 2218, 1868, 407, 293, 7126, 6399, 8872, 5907, 4182, 4067, 5609, 7676, 7754, 2410, 5458, 8909, 7375, 2849, 5583, 2946, 4197, 7114, 8453, 9050, 6784, 1915, 301, 9218, 8532, 4449, 9073, 3923, 6522, 2501, 4075, 2682, 4033, 3393, 6779, 4602, 492, 6037, 84, 2204, 209, 8657, 5023, 459, 9966, 193, 7335, 8863, 4354, 2754, 4572, 1575, 3555, 4657, 9403, 1407, 4207, 3803, 6414, 4179, 9789, 1778, 159, 5737, 405, 4880, 9355, 6925, 6557, 3454, 5735, 2554, 7120, 1064, 8738, 4848, 330, 9602, 4609, 1966, 9118, 8843, 2230, 1655, 5536, 504, 9324, 5207, 2599, 9984, 119, 4651, 2201, 9955, 7337, 2329, 5012, 1674, 9338, 9394, 8099, 8491, 7562, 2007, 5292, 4299, 5490, 1046, 6813, 4936, 9041, 4803, 7322, 5210, 5108, 6197, 101, 9703, 7708, 256, 1363, 637, 7532, 9645, 3784, 4874, 7637, 4829, 9436, 2079, 8976, 6407, 4937, 441, 9779, 8882, 2727, 9208, 9393, 4932, 6747, 2263, 7098, 5885, 5875, 2238, 5279, 5510, 7870, 286, 5197, 8345, 1024, 5611, 4122, 8898, 6339, 3201, 1192, 5684, 8019, 9631, 8229, 5759, 3329, 2283, 4648, 2137, 4371, 263, 951, 8178, 2055, 4976, 3856, 6079, 8889, 2202, 4383, 4155, 5063, 299, 5552, 6516, 3139, 9017, 4205, 2333, 1993, 7028, 9530, 7804, 8593, 6808, 4747, 8786, 1048, 8426, 1559, 4961, 2887, 1821, 4892, 2397, 2045, 1204, 5943, 5628, 4817, 9901, 6943, 9066, 3110, 7956, 7154, 6827, 6442, 7675, 8356, 1138, 2420, 175, 3499, 71, 1239, 6375, 1830, 5992, 6886, 1114, 3535, 6953, 7143, 1475, 3638, 3649, 2802, 1909, 6496, 2568, 9503, 4794, 2690, 8075, 1736, 2678, 8699, 1312, 7284, 5707, 1193, 1354, 5632, 4711, 2671, 9996, 2496, 2353, 1893, 1768, 7130, 1163, 6363, 3602, 3864, 1432, 451, 5201, 8815, 8676, 9491, 1203, 1628, 590, 3529, 4444, 8706, 6471, 1526, 6846, 9856, 7276, 1423, 7244, 5231, 9718, 3807, 694, 1112, 1789, 5769, 2463, 8008, 3406, 4879, 9909, 3652, 2273, 7069, 1714, 5720, 3153, 8029, 5703, 2877, 9294, 6212, 1211, 3552, 9568, 2888, 1989, 2935, 6381, 223, 8297, 7796, 8700, 1603, 6000, 4206, 1426, 3936, 1293, 2736, 96, 224, 2540, 3826, 4181, 4949, 7937, 5298, 9921, 1087, 9673, 348, 1580, 6857, 2657, 4230, 2317, 5425, 3150, 5951, 8199, 1079, 2510, 5378, 9175, 4138, 8973, 1119, 4660, 410, 1908, 6844, 3739, 5572, 5754, 3143, 775, 6427, 4410, 8185, 4095, 6470, 1854, 5768, 6762, 5525, 6598, 3443, 8319, 7487, 8602, 4156, 4189, 7979, 8122, 392, 7229, 3493, 7264, 8165, 3495, 8082, 3311, 2268, 631, 6196, 7062, 8631, 2697, 2564, 7281, 1005, 2796, 5013, 3365, 2288, 7201, 7236, 5534, 9072, 2375, 4089, 2941, 1137, 2211, 2918, 7601, 4083, 5692, 1869, 4446, 643, 4894, 740, 3989, 3342, 3828, 2572, 5920, 2227, 419, 3483, 9935, 5384, 5218, 4737, 5937, 533, 103, 8866, 8412, 6665, 4889, 623, 7984, 748, 4697, 4336, 7465, 5476, 6577, 3328, 2658, 6621, 1824, 4145, 8840, 1589, 793, 4390, 7341, 660, 9004, 9089, 1957, 3590, 5932, 8494, 8046, 4823, 398, 8837, 1247, 6798, 2093, 5702, 5070, 5373, 8365, 3162, 2251, 2477, 4050, 6690, 1113, 4124, 7092, 1704, 1806, 7267, 9184, 6139, 2164, 6104, 8658, 8038, 3994, 7966, 3679, 6991, 9594, 6923, 4319, 9608, 4554, 8242, 6281, 4288, 3138, 2332, 3632, 4127, 5728, 4119, 3506, 5026, 6211, 6499, 9999, 5587, 4942, 6159, 393, 722, 6974, 729, 5786, 1936]

In [ ]:
n_ensemble = 30

In [ ]:
y_collection = []

for s in seed_list:

    if len(y_collection) == n_ensemble:
        break

    print('initializing seed = ' +str(seed_list.index(s)))

    random.seed(s)
    np.random.seed(s)
    torch.manual_seed(s)

    turbo_m = TurboM(f=f,  # Handle to objective function
                     lb=f.lb,  # Numpy array specifying lower bounds
                     ub=f.ub,  # Numpy array specifying upper bounds
                     n_init=15,  # Number of initial bounds from an Symmetric Latin hypercube design
                     max_evals=1050,  # Maximum number of evaluations
                     n_trust_regions=2,  # Number of trust regions
                     batch_size=30,  # How large batch size TuRBO uses
                     verbose=True,  # Print information from each batch
                     use_ard=True,  # Set to true if you want to use ARD for the GP kernel
                     max_cholesky_size=2000,  # When we switch from Cholesky to Lanczos
                     n_training_steps=30,  # Number of steps of ADAM to learn the hypers
                     min_cuda=1024,  # Run on the CPU for small datasets
                     device="cpu",  # "cpu" or "cuda"
                     dtype="float64",  # float64 or float32
                    )
    
    turbo_m.optimize()
    
    y_collection.append(turbo_m.fX)
    print('Finished seed')

np.save('Penicilin_TurBO2.npy', y_collection)